In [1]:
# coding=utf-8                           #
# this is for coding in utf-8            #
####Caution###############################
# This code doesn't work in Web Compiler #
##########################################
#STEP 1
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen
import webbrowser
#from dateutil.parser import parse as p
from html_table_parser import parser_functions as parser

In [2]:
#STEP 2 
auth_key="6cd6e48399f4c36705662af1a99ba1acc67e0297" #authority key
#company_code="100250" #company code
#start_date='20170101'

In [3]:
def get_rcp(company_code, date='20170101'):
    
    if len(company_code) == 7: company_code = company_code[1:]
    print(company_code)

    url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd=" +company_code +"&start_dt="+ date +'&bsn_tp=A001'
    #STEP 4
    resultXML=urlopen(url)  #this is for response of XML
    result=resultXML.read() #Using read method
    #STEP 5
    xmlsoup=BeautifulSoup(result,'html.parser')
    #STEP 6
    data = pd.DataFrame()
    te=xmlsoup.findAll("list")

    for t in te:
        temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
            t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
            columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data=pd.concat([data,temp])
    
    if len(data) < 1 : return
    #STEP 7 

    data= data[data['rmk']!='정연']
    data = data[data['rcp_dt']<= '20181231']
    data['종목코드'] = company_code
    return data.reset_index(drop=True)

In [105]:
def detail(rcp_no= '20180402000599'):
    url2 = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+rcp_no
    #print(url2)
    page = BeautifulSoup(urlopen(url2).read(),'html.parser')
    #print(page)

    #print(body)
    #연결안씀
    #1. 감사보고서 중 손익계산서바로있음(독립된 손익계산서)
    
    #재무제표로 찾고 (비독립 손익계산서)
    #연결안씀
    body = str(page.find('head')).replace(u'\xa0', u' ')    
    '''
    v1은 그냥재무제표
    v2는 연결재무제표제외를 위한 " " 추가
    '''
    if len(body.split('재   무   제   표",')) == 1:
        if len(body.split('재  무  제  표",')) == 1:
            if len(body.split('재 무 제 표",')) == 1:
                if len(body.split('재무제표",')) <= 1:
                    print('재무제표 못찾음')
                else:
                    body = body.split('재무제표",')[1]
            else: 
                body = body.split('재 무 제 표",')[1]
        else:
            body = body.split('재  무  제  표",')[1]
    else:
        body = body.split('재   무   제   표",')[1]    
    
    body = body.split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
    body = [body[i][1:-1] for i in range(len(body))]
    #print(body)
    url3 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + body[0] + "&dcmNo=" + body[1] +"&eleId=" +body[2] + "&offset=" + body[3] +"&length=" + body[4] + "&dtd=dar3.xsd"
    print(url3)
    page = BeautifulSoup(urlopen(url3).read(),'html.parser')
    #print(page)
    #포괄손익계산서로 이동
    
    string_page = str(page.find('body')).replace(u'\xa0', u' ')
    
    if len(string_page.split('손   익   계   산   서')) == 1 :
        if len(string_page.split('손  익  계  산  서')) == 1 :
            if len(string_page.split('손 익 계 산 서')) == 1 : 
                if len(string_page.split('손익계산서')) <= 1 :
                    print('손익계산서 못찾음')
                else:
                    body = string_page.split('손익계산서')[1]
            else:
                body = string_page.split('손 익 계 산 서')[1]
        else:
            body = string_page.split('손  익  계  산  서')[1]
    else:
        body = string_page.split('손   익   계   산   서')[1]        
           
    body = BeautifulSoup(body, 'html.parser')
    
    #print(body)
    # 단위 추출하기
    if (body.find(align='RIGHT') != None) and '단위' in body.find(align="RIGHT").text:
        unit = body.find(align='RIGHT').text.replace(' ','').split('단위:')[1].split('원')[0]
        unit_dict = {'억': 100000000, '천만':10000000, '백만':1000000,'십만':100000,'만':10000,'천':1000, '':1}
        unit = unit_dict[unit]
    else: unit =1
    #테이블추출
    table = body.find_all('table')
    for i in range(len(table)):
        if '매출액' in str(table[i]):
            select_table = table[i]
            break
        elif '매출원가' in str(table[i]):
            select_table = table[i]
            break
        elif '당기순' in str(table[i]):
            select_table = table[i]
            break
        else:
            print('{}번째 테이블 없음'.format(i))

    
    p = parser.make2d(select_table)
    table = pd.DataFrame(p[1:], columns = p[0])
        
    #테이블가공
    if '주석' in table.columns : del table['주석']
    if '주 석' in table.columns : del table['주 석']
    if '주  석' in table.columns : del table['주  석']
    table.index = table.iloc[:,0]
    del table[table.columns[0]]
    table = table[table.applymap(lambda x : len(x)==0)==False]
    table = table.dropna(axis=1, how= 'all')
    table = table.fillna('0')
    table = table.replace('-','0')
    def comer(x):
        y = []
        x = x.replace('(','-').replace(')', '')
        for frag in x.split(',') :
            if frag[0] != '-' and len(frag) >3:
                frag = frag[0:3]
                y.append(frag)
                break
            y.append(frag)
        return ''.join(y)
    for i in range(len(table.columns)):
        table.iloc[:,i] = table.iloc[:,i].apply(lambda x : comer(x).split(' ')[0].split('\n')[0])
    #    table.iloc[:,i] = table.iloc[:,i].apply(lambda x : x.replace('-','0'))

    table = table.astype(dtype=np.int64)

    if (len(table.columns)> 2)&(len(table.columns) %2 == 0):
        keep = []
        for i in range(len(table.columns)//2):
            if table.columns[i*2] == table.columns[i*2 + 1]:
                table.iloc[:, i*2] = np.where(table.iloc[:, i*2]== 0 , table.iloc[:,i*2 + 1], table.iloc[:,i*2])
                keep.append(i*2)
        table = table.iloc[:,keep]
    
    #table = table.iloc[:,0]
    
    table = pd.DataFrame(table.iloc[:,0]).reset_index()
    dictofcolumns =  {table.columns[i] : i for i in range(len(table.columns))}
    table = table.rename(columns = dictofcolumns)
    
    table['단위'] = unit
    table['rcp_no'] = rcp_no
    
    return table


## 종목코드 rcp_no 추출

In [24]:
data_list = pd.read_excel('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\DART용7.xlsx',
                          sheet_name = 'Sheet7',
                          converters={'종목코드':str})

data_list = data_list[~data_list['종목코드'].isnull()]

In [25]:
data_list

,검색순서,계약자명,종목코드,검색키,rcp_no,rcp_no.1
0,2000,에스케이하이닉스주식회사,000660,NaN,NaN,NaN
5,2005,(주)만도,204320,NaN,NaN,NaN
10,2011,주식회사카프로,006380,NaN,NaN,NaN
15,2016,(주)경인양행,012610,NaN,NaN,NaN
20,2025,에스케이하이닉스주식회사,000660,NaN,NaN,NaN
27,2033,동원시스템즈(주),014820,NaN,NaN,NaN
28,2034,미래나노텍(주),095500,NaN,NaN,NaN
33,2040,(주)한솔케미칼,014680,NaN,NaN,NaN
38,2045,(주)휴비스,079980,NaN,NaN,NaN
42,2049,(주)삼양패키징,272550,NaN,NaN,NaN


In [26]:
result = pd.DataFrame([],columns = ["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk",'종목코드'])

In [27]:
for i in range(len(data_list)):
    df2 = get_rcp(data_list["종목코드"].iloc[i],'20170101')
    result = result.append(df2)

000660
204320
006380
012610
000660
014820
095500
014680
079980
272550
145990
005490
017810
034730
222800
004980
016800
018670
069620
000080
161390
012800
003620
003410
004490
005010


In [28]:
result.to_csv('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\종목코드rcp_2000억초과_extra.csv',mode='w',encoding = 'cp949')

In [29]:
result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk,종목코드
0,Y,SK하이닉스,000660,[기재정정]사업보고서 (2017.12),20180615000111,SK하이닉스,20180615,연,000660
1,Y,SK하이닉스,000660,사업보고서 (2016.12),20170331004537,SK하이닉스,20170331,연,000660
0,Y,만도,204320,[기재정정]사업보고서 (2017.12),20180814002255,만도,20180814,연,204320
1,Y,만도,204320,사업보고서 (2016.12),20170331004681,만도,20170331,연,204320
0,Y,카프로,006380,사업보고서 (2017.12),20180330001239,카프로,20180330,None,006380
1,Y,카프로,006380,사업보고서 (2016.12),20170331005159,카프로,20170331,None,006380
0,Y,경인양행,012610,사업보고서 (2017.12),20180330000601,경인양행,20180330,연,012610
1,Y,경인양행,012610,사업보고서 (2016.12),20170330000245,경인양행,20170330,연,012610
0,Y,SK하이닉스,000660,[기재정정]사업보고서 (2017.12),20180615000111,SK하이닉스,20180615,연,000660
1,Y,SK하이닉스,000660,사업보고서 (2016.12),20170331004537,SK하이닉스,20170331,연,000660


In [285]:
company_code = '100250'
date = '20170101'
df = get_rcp(company_code, date)
df.head()

100250


,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,진양홀딩스,100250,[기재정정]사업보고서 (2017.12),20180402000599,진양홀딩스,20180402,연
1,Y,진양홀딩스,100250,사업보고서 (2016.12),20170331000108,진양홀딩스,20170331,연


## rcp_no에해당하는 손익계산서추출

In [93]:
data_list = pd.read_excel('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\DART용7.xlsx',
                          sheet_name = 'Sheet7',converters={'rcp_no':str})

In [94]:
data_list.head(10)

,검색순서,계약자명,종목코드,검색키,rcp_no
0,2000,에스케이하이닉스주식회사,660.0,NaN,20180615000111
1,2001,현대그린파워(주),NaN,현대그린파워,20180404002585
2,2002,하나파워패키지(유),NaN,하나파워패키지,20180409001295
3,2003,아반스트레이트코리아(주),NaN,아반스트레이트코리아,20180704000199
4,2004,수완에너지주식회사,NaN,수완에너지,20180329001070
5,2005,(주)만도,204320.0,NaN,20180814002255
6,2006,코오롱바스프이노폼주식회사,NaN,코오롱바스프이노폼,20180328000914
7,2008,여천NCC(주),NaN,여천NCC,20180402000074
8,2010,경기그린에너지(주),NaN,경기그린에너지,20180404002769
9,2011,주식회사카프로,6380.0,NaN,20180330001239


In [96]:
data_list=data_list[~data_list['rcp_no'].isnull()]

In [103]:
result.to_csv('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\2000억초과_extra_연결제외재무제표.csv',mode='w',encoding = 'cp949')

In [101]:
result = pd.DataFrame([], columns =[0, 1, '단위','rcp_no'])

In [102]:
for i in range(len(data_list)):
    print(" ")
    print("---------------------------------------------------------")

    print("index : {}, 계약자명 : {} 시작" .format(i, data_list['계약자명'].iloc[i]))
    try:
        df2 = detail(data_list['rcp_no'].iloc[i])
        result = result.append(df2)
        print("index : {}, 계약자명 : {} 완료" .format(i, data_list['계약자명'].iloc[i]))
    except TypeError as e:
        print("error : {}, 오류".format(e))
    except ValueError as v:
        print("error : {}, 오류".format(v))
    except UnboundLocalError as u:
        print("error : {}, 오류".format(u))

 
---------------------------------------------------------
index : 0, 계약자명 : 에스케이하이닉스주식회사 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180615000111&dcmNo=6217166&eleId=17&offset=959474&length=67317&dtd=dar3.xsd
index : 0, 계약자명 : 에스케이하이닉스주식회사 완료
 
---------------------------------------------------------
index : 1, 계약자명 : 현대그린파워(주) 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180404002585&dcmNo=6072895&eleId=ul&offset=ul&length=ul&dtd=dar3.xsd
손익계산서 못찾음
error : expected string or bytes-like object, 오류
 
---------------------------------------------------------
index : 2, 계약자명 : 하나파워패키지(유) 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180409001295&dcmNo=6087280&eleId=ul&offset=ul&length=ul&dtd=dar3.xsd
손익계산서 못찾음
error : expected string or bytes-like object, 오류
 
---------------------------------------------------------
index : 3, 계약자명 : 아반스트레이트코리아(주) 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180704000199&dcmNo=6238178&eleId=ul&offset=u

0번째 테이블 없음
error : invalid literal for int() with base 10: '90원', 오류
 
---------------------------------------------------------
index : 32, 계약자명 : (주)한솔케미칼 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180326000391&dcmNo=6015594&eleId=15&offset=834184&length=102295&dtd=dar3.xsd
index : 32, 계약자명 : (주)한솔케미칼 완료
 
---------------------------------------------------------
index : 33, 계약자명 : 동우화인켐(주) 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180712000341&dcmNo=6248924&eleId=ul&offset=ul&length=ul&dtd=dar3.xsd
손익계산서 못찾음
error : expected string or bytes-like object, 오류
 
---------------------------------------------------------
index : 34, 계약자명 : (주)대륜발전 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180411001568&dcmNo=6100909&eleId=ul&offset=ul&length=ul&dtd=dar3.xsd
손익계산서 못찾음
error : expected string or bytes-like object, 오류
 
---------------------------------------------------------
index : 35, 계약자명 : 한국실리콘(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpN

http://dart.fss.or.kr/report/viewer.do?rcpNo=20180330001796&dcmNo=6033870&eleId=15&offset=1098146&length=125642&dtd=dar3.xsd
index : 66, 계약자명 : 한국남부발전(주) 완료
 
---------------------------------------------------------
index : 67, 계약자명 : 한국중부발전(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190516000044&dcmNo=6741815&eleId=17&offset=1094805&length=121283&dtd=dar3.xsd
index : 67, 계약자명 : 한국중부발전(주) 완료
 
---------------------------------------------------------
index : 68, 계약자명 : 씨제이제일제당(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180406002292&dcmNo=6083972&eleId=17&offset=1240630&length=74664&dtd=dar3.xsd
index : 68, 계약자명 : 씨제이제일제당(주) 완료
 
---------------------------------------------------------
index : 69, 계약자명 : 동서식품(주) 시작
 재무제표 못찾음
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180326000399&dcmNo=6015617&eleId=ul&offset=ul&length=ul&dtd=dar3.xsd
손익계산서 못찾음
error : expected string or bytes-like object, 오류
 
---------------------------------------------------------
index : 7

In [108]:
result.to_csv('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\2000억초과_extra_연결재무제표.csv',mode='w',encoding = 'cp949')

In [106]:
result = pd.DataFrame([], columns =[0, 1, '단위','rcp_no'])

In [107]:
for i in range(len(data_list)):
    print(" ")
    print("---------------------------------------------------------")

    print("index : {}, 계약자명 : {} 시작" .format(i, data_list['계약자명'].iloc[i]))
    try:
        df2 = detail(data_list['rcp_no'].iloc[i])
        result = result.append(df2)
        print("index : {}, 계약자명 : {} 완료" .format(i, data_list['계약자명'].iloc[i]))
    except TypeError as e:
        print("error : {}, 오류".format(e))
    except ValueError as v:
        print("error : {}, 오류".format(v))
    except UnboundLocalError as u:
        print("error : {}, 오류".format(u))

 
---------------------------------------------------------
index : 0, 계약자명 : 에스케이하이닉스주식회사 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180615000111&dcmNo=6217166&eleId=15&offset=452930&length=82780&dtd=dar3.xsd
index : 0, 계약자명 : 에스케이하이닉스주식회사 완료
 
---------------------------------------------------------
index : 1, 계약자명 : 현대그린파워(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180404002585&dcmNo=6072895&eleId=3&offset=11564&length=223176&dtd=dar3.xsd
0번째 테이블 없음
index : 1, 계약자명 : 현대그린파워(주) 완료
 
---------------------------------------------------------
index : 2, 계약자명 : 하나파워패키지(유) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180409001295&dcmNo=6087280&eleId=3&offset=7037&length=238591&dtd=dar3.xsd
0번째 테이블 없음
index : 2, 계약자명 : 하나파워패키지(유) 완료
 
---------------------------------------------------------
index : 3, 계약자명 : 아반스트레이트코리아(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180704000199&dcmNo=6238178&eleId=3&offset=10598&length=194271&dtd=dar3.xsd
0번째 테이블 없음
index : 3,

0번째 테이블 없음
index : 31, 계약자명 : 에스케이실트론(주) 완료
 
---------------------------------------------------------
index : 32, 계약자명 : (주)한솔케미칼 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180326000391&dcmNo=6015594&eleId=13&offset=233493&length=108827&dtd=dar3.xsd
index : 32, 계약자명 : (주)한솔케미칼 완료
 
---------------------------------------------------------
index : 33, 계약자명 : 동우화인켐(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180712000341&dcmNo=6248924&eleId=3&offset=9811&length=373270&dtd=dar3.xsd
0번째 테이블 없음
index : 33, 계약자명 : 동우화인켐(주) 완료
 
---------------------------------------------------------
index : 34, 계약자명 : (주)대륜발전 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180411001568&dcmNo=6100909&eleId=3&offset=12145&length=240629&dtd=dar3.xsd
index : 34, 계약자명 : (주)대륜발전 완료
 
---------------------------------------------------------
index : 35, 계약자명 : 한국실리콘(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402000220&dcmNo=6043629&eleId=13&offset=156173&length=130&dtd=dar3.xsd
손익

index : 64, 계약자명 : 도레이첨단소재주식회사 완료
 
---------------------------------------------------------
index : 65, 계약자명 : 코스포영남파워주식회사 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180330002476&dcmNo=6036436&eleId=3&offset=11390&length=213144&dtd=dar3.xsd
error : invalid literal for int() with base 10: '후속적', 오류
 
---------------------------------------------------------
index : 66, 계약자명 : 한국남부발전(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180330001796&dcmNo=6033870&eleId=13&offset=357898&length=139537&dtd=dar3.xsd
index : 66, 계약자명 : 한국남부발전(주) 완료
 
---------------------------------------------------------
index : 67, 계약자명 : 한국중부발전(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20190516000044&dcmNo=6741815&eleId=15&offset=384940&length=168671&dtd=dar3.xsd
index : 67, 계약자명 : 한국중부발전(주) 완료
 
---------------------------------------------------------
index : 68, 계약자명 : 씨제이제일제당(주) 시작
http://dart.fss.or.kr/report/viewer.do?rcpNo=20180406002292&dcmNo=6083972&eleId=15&offset=410057&length

In [174]:
rcp_no = data_list['rcp_no'].iloc[45]
url2 = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+rcp_no
#print(url2)
page = BeautifulSoup(urlopen(url2).read(),'html.parser')
#print(page)
body = str(page.find('head')).replace(u'\xa0', u' ')    

In [172]:
print(body)

<head>
<title>
SKC/사업보고서/2018.04.02
</title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<style type="text/css">
    .x-window-dlg .ext-mb-download {
        background:transparent url(images/download.gif) no-repeat top left;
        height:46px;
    }
</style>
<link charset="utf-8" href="/css/viewer.css" rel="stylesheet" type="text/css"/>
<script src="/js/prototype.js" type="text/javascript"></script>
<script src="/js/jquery/jquery-all.js" type="text/javascript"></script>
<!-- 2014.03.11 ext 3.4 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/css/ext-all-ie8.css" ><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><link href="/js/ext-viewer/resources/css/ext-all.css" rel="stylesheet" type="text/css"/><!--<![endif]-->
<script src="/js/ext-viewer/adapter/ext/ext-base.js" type="text/javascript"></script>
<script src="/js/ext-viewer/ext-all.js" type="text/javascript">

In [175]:
if len(body.split(' 재   무   제   표",')) == 1:
    if len(body.split(' 재  무  제  표",')) == 1:
        if len(body.split(' 재 무 제 표",')) == 1:
            if len(body.split(' 재무제표",')) <= 1:
                print(' 재무제표 못찾음')
            else:
                body = body.split(' 재무제표",')[1]
        else: 
            body = body.split(' 재 무 제 표",')[1]
    else:
        body = body.split(' 재  무  제  표",')[1]
else:
    body = body.split(' 재   무   제   표",')[1]    

body = body.split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
body = [body[i][1:-1] for i in range(len(body))]
#print(body)
url3 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + body[0] + "&dcmNo=" + body[1] +"&eleId=" +body[2] + "&offset=" + body[3] +"&length=" + body[4] + "&dtd=dar3.xsd"
print(url3)
page = BeautifulSoup(urlopen(url3).read(),'html.parser')
#print(page)

http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402002999&dcmNo=6051794&eleId=15&offset=164372&length=66822&dtd=dar3.xsd


In [163]:
string_page = str(page.find('body')).replace(u'\xa0', u' ')

if len(string_page.split('손   익   계   산   서')) == 1 :
    if len(string_page.split('손  익  계  산  서')) == 1 :
        if len(string_page.split('손 익 계 산 서')) == 1 : 
            if len(string_page.split('손익계산서')) <= 1 :
                print('손익계산서 못찾음')
            else:
                body = string_page.split('손익계산서')[1]
        else:
            body = string_page.split('손 익 계 산 서')[1]
    else:
        body = string_page.split('손  익  계  산  서')[1]
else:
    body = string_page.split('손   익   계   산   서')[1]      

In [164]:
body = BeautifulSoup(body, 'html.parser')
#print(body)
# 단위 추출하기
if (body.find(align='RIGHT') != None) and '단위' in body.find(align="RIGHT").text:
    unit = body.find(align='RIGHT').text.replace(' ','').split('단위:')[1].split('원')[0]
    unit_dict = {'억': 100000000, '천만':10000000, '백만':1000000,'십만':100000,'만':10000,'천':1000, '':1}
    unit = unit_dict[unit]
else: unit =1
#테이블추출
table = body.find_all('table')
for i in range(len(table)):
    if '매출액' in str(table[i]):
        select_table = table[i]
        break
    elif '매출원가' in str(table[i]):
        select_table = table[i]
        break
    elif '당기순' in str(table[i]):
        select_table = table[i]
        break
    else:
        print('{}번째 테이블 없음'.format(i))

p = parser.make2d(select_table)
table = pd.DataFrame(p[1:], columns = p[0])

In [165]:
table

,,제 45 기,제 44 기,제 43 기
0,수익(매출액),"1,403,473,050,602","1,380,829,845,672","1,616,227,686,728"
1,매출원가,"1,126,493,164,233","1,112,046,825,140","1,239,873,115,012"
2,매출총이익,"276,979,886,369","268,783,020,532","376,354,571,716"
3,판매비와관리비,"137,731,278,809","170,970,756,220","174,095,677,484"
4,영업이익(손실),"139,248,607,560","97,812,264,312","202,258,894,232"
5,기타이익,"17,352,709,506","67,339,586,665","398,795,732,934"
6,기타손실,"51,595,370,275","12,806,736,439","221,189,592,121"
7,금융수익,"12,666,985,824","18,867,340,197","37,313,307,910"
8,금융원가,"37,024,522,042","49,950,960,997","51,154,033,983"
9,법인세비용차감전순이익(손실),"80,648,410,573","121,261,493,738","366,024,308,972"


In [145]:
'후속적' in table

False

In [152]:
#테이블가공
if '주석' in table.columns : del table['주석']
table.index = table.iloc[:,0]
del table[table.columns[0]]
table = table[table.applymap(lambda x : len(x)==0)==False]
table = table.dropna(axis=1, how= 'all')
table = table.fillna('0')
table = table.replace('-','0')
def comer(x):
    y = []
    x = x.replace('(','-').replace(')', '')
    for frag in x.split(',') :
        if frag[0] != '-' and len(frag) >3:
            frag = frag[0:3]
            y.append(frag)
            break
        y.append(frag)
    return ''.join(y)
for i in range(len(table.columns)):
    table.iloc[:,i] = table.iloc[:,i].apply(lambda x : comer(x).split(' ')[0].split('\n')[0])
#    table.iloc[:,i] = table.iloc[:,i].apply(lambda x : x.replace('-','0'))
table = table.astype(dtype=np.int64)
if (len(table.columns)> 2)&(len(table.columns) %2 == 0):
    keep = []
    for i in range(len(table.columns)//2):
        if table.columns[i*2] == table.columns[i*2 + 1]:
            table.iloc[:, i*2] = np.where(table.iloc[:, i*2]== 0 , table.iloc[:,i*2 + 1], table.iloc[:,i*2])
            keep.append(i*2)
    table = table.iloc[:,keep]

ValueError: invalid literal for int() with base 10: '후속적'

# 명칭 정제 및 번호 가져오기

In [5]:
from selenium import webdriver      #셀레니움 기본
from selenium.webdriver.support.ui import Select        #콤보박스 선택용
from selenium.webdriver.support.ui import WebDriverWait  #팝업용1 
from selenium.webdriver.support import expected_conditions as EC #팝업용2
from selenium.common.exceptions import TimeoutException #팝업용3
from selenium.webdriver.common.alert import Alert    #팝업창
from bs4 import BeautifulSoup   #해당페이지 parse

import urllib.request   #  attrs 사용
import re           # 정규표현식
import time         # 쉴때사용
import pandas as pd #

In [6]:
browser = webdriver.Chrome("C:\\Users\\312263\\Desktop\\python\\chromedriver_v3")
browser.get("http://dart.fss.or.kr/dsab001/main.do?autoSearch=true#")
browser.implicitly_wait(1)

In [31]:
data_list = pd.read_excel('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\DART용7_1.xlsx',
                          sheet_name = 'Sheet7',
                          converters={'종목코드':str})


In [32]:
data_list.head()

,검색순서,계약자명,종목코드,검색키,rcp_no,rcp_no.1
0,2000,에스케이하이닉스주식회사,000660,NaN,NaN,NaN
1,2001,현대그린파워(주),NaN,현대그린파워,NaN,NaN
2,2002,하나파워패키지(유),NaN,하나파워패키지,NaN,NaN
3,2003,아반스트레이트코리아(주),NaN,아반스트레이트코리아,NaN,NaN
4,2004,수완에너지주식회사,NaN,수완에너지,NaN,NaN


In [10]:
data_list = data_list[~data_list['검색키'].isnull()]

In [51]:
data_list = data_list[data_list['rcp_no'].isnull()]

In [12]:
data_list

,검색순서,계약자명,종목코드,검색키,rcp_no,rcp_no.1
1,2001,현대그린파워(주),NaN,현대그린파워,NaN,NaN
2,2002,하나파워패키지(유),NaN,하나파워패키지,NaN,NaN
3,2003,아반스트레이트코리아(주),NaN,아반스트레이트코리아,NaN,NaN
4,2004,수완에너지주식회사,NaN,수완에너지,NaN,NaN
6,2006,코오롱바스프이노폼주식회사,NaN,코오롱바스프이노폼,NaN,NaN
7,2007,대림산업(주)석유화학사업부,NaN,대림산업석유화학사업부,NaN,NaN
8,2008,여천NCC(주),NaN,여천NCC,NaN,NaN
9,2010,경기그린에너지(주),NaN,경기그린에너지,NaN,NaN
11,2012,포천파워(주),NaN,포천파워,NaN,NaN
12,2013,춘천에너지주식회사,NaN,춘천에너지,NaN,NaN


In [11]:
def search2(browser,ind= 1, search_name= '서경타니골프앤리조트'):
    
    e = browser.find_element_by_css_selector("#textCrpNm")
    e.clear()
    time.sleep(1)
    e.send_keys(search_name) # 주소명
    time.sleep(1)
    browser.find_element_by_css_selector("#searchpng").click()
    time.sleep(1.5)
    
    res = browser.page_source
    soup = BeautifulSoup(res, "html.parser")

    body = soup.select('#listContents > div.table_list > table')
    p = parser.make2d(body[0])
    table = pd.DataFrame(p[1:], columns = p[0])

    li = str(body[0]).split('onclick="openReportViewer(')
    rcp_no_list = []

    for i in range(1,len(li)):
        rcp_no = li[i].split(')')[0].split(",'")[0]
        rcp_no_list.append(rcp_no)
    table['rcp_no'] = rcp_no_list
    table['검색순서'] = ind
    table['검색키'] = search_name
    
    return table

In [42]:
result = pd.DataFrame([],columns = ['번호','공시대상회사','보고서명','제출인','접수일자','rcp_no','검색순서','검색키'])

In [86]:
ind = 2089

In [87]:
search_name = '동서석유화학'

In [91]:
res = browser.page_source
soup = BeautifulSoup(res, "html.parser")

body = soup.select('#listContents > div.table_list > table')
p = parser.make2d(body[0])
table = pd.DataFrame(p[1:], columns = p[0])

li = str(body[0]).split('onclick="openReportViewer(')
rcp_no_list = []

for i in range(1,len(li)):
    rcp_no = li[i].split(')')[0].split(",'")[0]
    rcp_no_list.append(rcp_no)
table['rcp_no'] = rcp_no_list
table['검색순서'] = ind
table['검색키'] = search_name
#result = result.append(table)

In [92]:
table

,번호,공시대상회사,보고서명,제출인,접수일자,비고,rcp_no,검색순서,검색키
0,1,동원홈푸드,감사보고서 \n\t\t\t\t\t\t\t\t(2018.12),한영회계법인,2019.03.13,,'20190313000622',2089,동서석유화학
1,2,동원홈푸드,감사보고서 \n\t\t\t\t\t\t\t\t(2017.12),한영회계법인,2018.03.13,,'20180313000381',2089,동서석유화학
2,3,동원홈푸드,감사보고서 \n\t\t\t\t\t\t\t\t(2016.12),한영회계법인,2017.03.21,,'20170321000045',2089,동서석유화학


In [89]:
result.tail(10)

,rcp_no,검색순서,검색키,공시대상회사,번호,보고서명,비고,접수일자,제출인
0,'20190401004092',2086,CJ제일제당,CJ제일제당,1,사업보고서 \n\t\t\t\t\t\t\t\t(2018.12),,2019.04.01,CJ제일제당
1,'20180406002292',2086,CJ제일제당,CJ제일제당,2,[기재정정]사업보고서 \n\t\t\t\t\t\t\t\t(2017.12),,2018.04.06,CJ제일제당
2,'20180330003168',2086,CJ제일제당,CJ제일제당,3,사업보고서 \n\t\t\t\t\t\t\t\t(2017.12),,2018.03.30,CJ제일제당
3,'20170331004895',2086,CJ제일제당,CJ제일제당,4,사업보고서 \n\t\t\t\t\t\t\t\t(2016.12),,2017.03.31,CJ제일제당
0,'20190618000300',2081,도레이첨단소재,도레이첨단소재,1,감사보고서 \n\t\t\t\t\t\t\t\t(2019.03),,2019.06.18,한영회계법인
1,'20180621000437',2081,도레이첨단소재,도레이첨단소재,2,감사보고서 \n\t\t\t\t\t\t\t\t(2018.03),,2018.06.21,한영회계법인
2,'20170620000011',2081,도레이첨단소재,도레이첨단소재,3,감사보고서 \n\t\t\t\t\t\t\t\t(2017.03),,2017.06.20,한영회계법인
0,'20190702000170',2089,동서석유화학,동서석유화학,1,감사보고서 \n\t\t\t\t\t\t\t\t(2019.03),,2019.07.02,삼일회계법인
1,'20180702000514',2089,동서석유화학,동서석유화학,2,감사보고서 \n\t\t\t\t\t\t\t\t(2018.03),,2018.07.02,삼일회계법인
2,'20170707000166',2089,동서석유화학,동서석유화학,3,감사보고서 \n\t\t\t\t\t\t\t\t(2017.03),,2017.07.07,삼일회계법인


In [18]:
data_list.loc[:52].shape

(40, 6)

In [20]:
data_list.iloc[40]

검색순서            2063
계약자명        (주)동원홈푸드
종목코드             NaN
검색키            동원홈푸드
rcp_no           NaN
rcp_no.1         NaN
Name: 53, dtype: object

In [15]:
data_list[data_list['검색키']== '포스코에이에스티']

,검색순서,계약자명,종목코드,검색키
8,2054,(주)포스코에이에스티,NaN,포스코에이에스티


In [22]:
for i in range(40,len(data_list)):
    df2 = search2(browser, data_list["검색순서"].iloc[i], data_list["검색키"].iloc[i])
    result = result.append(df2)

In [90]:
result.to_csv('C:\\Users\\312263\\Desktop\\python\\01. pkg\\03. 크롤링\\01. dart\\명칭rcp_2000억초과_extra_수기.csv',mode='w',encoding = 'cp949')